<div class="bg-primary text-lg-left text-white">
    <div >
        <h1 class="text-center" style="padding: 10px 0 10px 0;">
          Benchmark wor2vec
        </h1>
    </div>
</div>

- **[approche classique](#ap)**

- **[Vectorisation de notre corpus](#w2)**

- **[Creation des features word2vec](#cr)**

- **[intersection entre le vocab google news externe et notre vocab](#go)**

- **[intersection entre le vocab wikipedia externe et notre vocab](#wi)**

- **[intersection entre le vocab conceptNet externe et notre vocab](#co)**

- **[intersection entre le vocab fastText externe et notre vocab](#fa)**

Nous allons cette fois plutot que de prendre comme features le tf-idf de notre corpus , nous utiliserons word2vec pour generer nos nouvelles features.

Nous essayerons ensuite d enrichir notre vocab avec des vocabs externe disponible sur le github de gensimm 
Nous testerons les vocabs wikipedia, googlenews...

https://github.com/RaRe-Technologies/gensim-data/releases

En esperant pouvoir ameliorer nos predictions



In [2]:
# Import Python libraries
import os
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import re
import unicodedata
from bs4 import BeautifulSoup
# from langdetect import detect
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

from IPython.core.display import display, HTML
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
import timeit
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score
import sklearn.metrics as metrics

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler

In [ ]:
# pip uninstall -y gensim

Found existing installation: gensim 3.2.0
Uninstalling gensim-3.2.0:
  Successfully uninstalled gensim-3.2.0


In [3]:
from google.colab import drive
drive.mount('/content/ShardDrives/', force_remount=True)

Mounted at /content/ShardDrives/


In [4]:
data = pd.read_csv("/content/ShardDrives/MyDrive/nlp/stackoverflow_cleaned.csv",sep=";", index_col=0,
                               converters={
                                   "tags": literal_eval,
                                   "text": literal_eval
                               }
                  )

data.head()

,tags,full_text,text,score
Id,,,,
5767325,"[javascript, arrays]",How can I remove a specific item from an array...,"[item, array, array, number, method, element, ...",10385
7244321,[git],How do I update or sync a forked repository on...,"[repository, github, project, fix, pull, reque...",4359
8710619,[java],"Why don't Java's +=, -=, *=, /= compound assig...","[java, compound, assignment, operator, today, ...",3779
6245570,[git],How to get the current branch name in Git? I'm...,"[branch, name, git, subversion, background, br...",3372
8467424,[bash],Echo newline in Bash prints literal \n In Bash...,"[echo, newline, bash, print, bash, newline, ne...",3050


Test simple dans un premier temp avec un modele tfidf qui nous servira de baseline , que l on compareras avec notre modele  word2vec

In [5]:
X = data[["text"]]
# X = data["text"]
y = data['tags']

multilabel_binarizer = MultiLabelBinarizer()
y_encoded = multilabel_binarizer.fit_transform(y)

tfidf = Pipeline([
                ('tfidf', TfidfVectorizer(analyzer="word",
                             max_df=.6,
                             min_df=0.005,
                             tokenizer=None,
                             preprocessor=' '.join,
                             stop_words=None,
                             lowercase=False,
                             ngram_range=(1, 3)
                                         ))
            ])

X_train, X_test, y_train , y_test = train_test_split(X , y_encoded, test_size = 0.2, random_state= 0)

In [6]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
# gensim.downloader.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
# gensim.downloader.load('word2vec-google-news-300')

Metriques

In [7]:
def getMetrics(model,name,X_test, y_test):
    frames=[]
    frame = pd.DataFrame()
    y_pred = model.predict(X_test) 
    frame['estimator'] = [name]  
    frame['accuracy'] = np.round((accuracy_score(y_test, y_pred)), 5)
    frame['f1_macro'] = np.round(f1_score(y_test, y_pred, average="macro"), 3)
    frame['f1_micro'] = np.round(f1_score(y_test, y_pred, average="micro"), 3)
    frame['jaccard'] = np.round(jaccard_score(y_test, y_pred, average="weighted"), 3)
    frame['recall'] = np.round(recall_score(y_test, y_pred, average="weighted"), 3)
    frame['precision'] = np.round(precision_score(y_test, y_pred, average="weighted"), 3)                            
    frames.append(frame)
    return pd.concat(frames) 



<a name='ap'></a>

<div class="bg-primary text-lg-left text-white">
    <div >
        <h1 class="text-center" style="padding: 10px 0 10px 0;">
          approche classique
        </h1>
    </div>
</div>

In [7]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [8]:
tfidf.fit(X_train['text'])
feat_train = tfidf.transform(X_train['text'])
feat_test = tfidf.transform(X_test['text'])
print(feat_train.shape,feat_test.shape)

(74960, 644) (18741, 644)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50)
clf.fit(feat_train[0:10000], y_train[0:10000])

RandomForestClassifier(n_estimators=50)

In [ ]:
getMetrics(clf,'RandomForestClassifier',feat_test, y_test)

,estimator,accuracy,f1_macro,f1_micro,jaccard,recall,precision
0,RandomForestClassifier,0.31498,0.392,0.528,0.357,0.38,0.86


In [ ]:
logistic =  Pipeline([
    ('test',OneVsRestClassifier(LogisticRegression(max_iter=500))), 
])
logistic.fit(feat_train, y_train)

Pipeline(steps=[('test',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=500)))])

In [ ]:
getMetrics(logistic,'logistic',feat_test, y_test)

,estimator,accuracy,f1_macro,f1_micro,jaccard,recall,precision
0,logistic,0.32175,0.493,0.579,0.411,0.45,0.783


tags list

In [10]:
# !pip install  gensim===4.1.1

     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0



  <a name='w2'></a>

<div class="bg-primary text-lg-left text-white">
    <div >
        <h1 class="text-center" style="padding: 10px 0 10px 0;">
          Vectorisation de notre corpus
        </h1>
    </div>
</div>



Word2vec est un groupe de modèles utilisé pour le plongement lexica
Word2vec est une sorte d’ACP non linéaire en ce sens qu’il réduit les dimensions
word2vec est plus d’une ACP non linéaire car il prend en compte le contexte mais ne s’en éloigne pas tant que ce ça.

Ce sont des réseaux de neurones artificiels à deux couches entraînés pour reconstruire le contexte linguistique des mots.


Deux architectures ont été initialement proposées pour apprendre les Word2vec, le modèle de sacs de mots continus (CBOW: continuous bag of words) et le modèle skip-gram. Le CBOW vise à prédire un mot étant donné son contexte, c'est-à-dire étant donné les mots qui en sont proches dans le texte. Un tel contexte est par exemple les 5 mots à droite et les 5 mots à gauche du mot à prédire. Le skip-gram a une architecture symétrique visant à prédire les mots du contexte étant donné un mot en entrée.

les modèles CBOW et Skip-gram de word2vec.

En pratique, le modèle CBOW est plus rapide à apprendre, mais le modèle skip-gram donne généralement de meilleurs résultats.

Dans les deux cas, le réseau de neurones comporte deux couches. La couche cachée contient quelques centaines de neurones et constitue, à l'issue de la représentation, le plongement lexical (embedding) permettant de représenter un mot. La couche de sortie permet d'implémenter une tâche de classification au moyen d'une softmax.

L'apprentissage ne nécessite néanmoins aucun label, la vérité terrain étant directement déduite des données et plus particulièrement de la proximité des mots au sein du corpus d'entraînement. En ce sens, l'apprentissage de Word2vec constitue un apprentissage auto-supervisé.

In [8]:
import multiprocessing
from gensim.models import Word2Vec


model = Word2Vec(
                X_train["text"], 
                vector_size=300, 
                window=20,
                min_count=2, 
                sample=6e-5, 
                alpha=0.03, 
                min_alpha=0.0007, 
                negative=20,                          

                workers=7)



model.corpus_count

74960

In [9]:
model.epochs

5

In [ ]:
model.build_vocab(X_train['text'])
# model.train(X_train['text'], total_examples=model.corpus_count, epochs=model.epochs, report_delay=1)
model.train(X_train['text'], total_examples=model.corpus_count, epochs=30, report_delay=1)

(4610082, 11387685)

In [10]:
len(model.wv)

28409

In [11]:
model.wv.get_vecattr("java", "count") 

10701

In [12]:
model.wv.get_vecattr("c#", "count") 

5359

In [13]:
model.wv.key_to_index["java"]

26

Test de la vectorisation des mots

In [14]:
model.wv.most_similar(positive=["linux"])

[('operating', 0.8517012596130371),
 ('arch', 0.8257924318313599),
 ('gnu', 0.8211150765419006),
 ('debian', 0.8118247985839844),
 ('bsd', 0.8115512132644653),
 ('os', 0.782404363155365),
 ('syscalls', 0.7822757363319397),
 ('glibc', 0.7794502973556519),
 ('ubuntu', 0.7740756273269653),
 ('nfs', 0.773507297039032)]

In [15]:
model.wv.most_similar(positive=["problem"])

[('idea', 0.5939031839370728),
 ('exceed', 0.5400220155715942),
 ('pruning', 0.5215069055557251),
 ('die', 0.5203868746757507),
 ('wordlist', 0.5193285346031189),
 ('solution', 0.5140413045883179),
 ('anyone', 0.5125587582588196),
 ('tower', 0.5046101212501526),
 ('o(min(log', 0.5041437745094299),
 ('n(number', 0.5006166696548462)]

In [16]:
model.wv.most_similar(positive=["sql"])

[('ssms', 0.8773359060287476),
 ('mssql', 0.840709388256073),
 ('sp', 0.8260722756385803),
 ('dbms', 0.8226304054260254),
 ('ddl', 0.7937182188034058),
 ('query', 0.7922891974449158),
 ('procedure', 0.7903407216072083),
 ('resultset', 0.7863214612007141),
 ('subquery', 0.7855039238929749),
 ('varchar', 0.7833269238471985)]

In [17]:
model.wv.most_similar(positive=["database"])

[('db', 0.8428206443786621),
 ('oltp', 0.7693713307380676),
 ('rdbms', 0.7633110284805298),
 ('sql', 0.7512076497077942),
 ('nosql', 0.7313162684440613),
 ('dbms', 0.7286774516105652),
 ('record', 0.7282244563102722),
 ('phpmyadmin', 0.7086646556854248),
 ('mssql', 0.7073016166687012),
 ('mdb', 0.70614093542099)]

In [18]:
model.save('trainedOwnWords_word2vec.bin')

**Les dix premières coordonnées du vecteur associé au mot java.**

In [19]:
model.wv['php'].shape, model.wv['php'][:10]

((300,),
 array([-0.3831823 , -0.15032849, -0.59266686, -0.03302992,  1.5276675 ,
        -0.82867527,  0.48077607,  0.6954574 ,  0.18765399, -0.59474945],
       dtype=float32))



  <a name='cr'></a>

<div class="bg-primary text-lg-left text-white">
    <div >
        <h1 class="text-center" style="padding: 10px 0 10px 0;">
          Creation des features word2vec
        </h1>
    </div>
</div>




### Pour chaque phrase, on fait la somme des vecteurs associés aux mots qui la composent ou pas si le mot n’est pas dans le vocabulaire

In [20]:
import numpy

def get_vect(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return numpy.zeros((model.vector_size,))

def sum_vectors(phrase, model):
    return sum(get_vect(w, model) for w in phrase)

def word2vec_features(X, model):
    feats = numpy.vstack([sum_vectors(p, model) for p in X])
    return feats

wv_train_feat = word2vec_features(X_train["text"], model)
wv_train_feat.shape

(74960, 300)

In [21]:
wv_test_feat = word2vec_features(X_test["text"], model)

In [22]:
wv_train_feat.shape

(74960, 300)

In [23]:
wv_test_feat.shape

(18741, 300)

In [24]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(wv_train_feat[0:10000], y_train[0:10000])

RandomForestClassifier()

In [25]:
getMetrics(clf,'RandomForestWV',wv_test_feat, y_test)

,estimator,accuracy,f1_macro,f1_micro,jaccard,recall,precision
0,RandomForestWV,0.19487,0.217,0.378,0.23,0.24,0.921


Notre modele Word2vec est legerement superieur au modele tf idf

In [27]:
logisticWv =  Pipeline([
    #with word to vector 
    ('std',StandardScaler()),
    ('test',OneVsRestClassifier(LogisticRegression(max_iter=1000))), 
])
logisticWv.fit(wv_train_feat, y_train)
getMetrics(logisticWv,'logisticWv',wv_test_feat, y_test)
# logisticWv.score(wv_test_feat, y_test)

,estimator,accuracy,f1_macro,f1_micro,jaccard,recall,precision
0,logisticWv,0.3575,0.514,0.622,0.456,0.504,0.779


In [29]:
logisticTFidf =  Pipeline([
    #with tf idf        
    ('tfidf', tfidf),
    ('std',StandardScaler(with_mean=False)),
    ('test',OneVsRestClassifier(LogisticRegression(max_iter=500))), 
])
logisticTFidf.fit(X_train['text'], y_train)
getMetrics(logisticTFidf,'logisticTFIDF',X_test['text'], y_test)
# logisticWv.score(X_test['text'], y_test)

,estimator,accuracy,f1_macro,f1_micro,jaccard,recall,precision
0,logisticTFIDF,0.34705,0.543,0.609,0.443,0.5,0.758


<a name='go'></a>

<div class="bg-primary text-lg-left text-white">
    <div >
        <h1 class="text-center" style="padding: 10px 0 10px 0;">
          intersection entre le vocab google news externe et notre vocab
        </h1>
    </div>
</div>


Nous allons encore essayer d ameliorer notre modele en y integrant d dautre vocab glaner sur internet

- googleNEws
- wikipedia
- conceptnet

ces vocab seront 'mixer' avec notre propre modele word2vec, nous lancerons ensuite un entrainement pour comparer les resultats

In [ ]:
# import gensim.downloader
# gensim.downloader.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [30]:
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors

In [ ]:
# google_wv = gensim.models.KeyedVectors.load_word2vec_format('wv_googlenews/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [31]:
google_wv = gensim.models.KeyedVectors.load_word2vec_format('/content/ShardDrives/MyDrive/nlp/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
# google_wv.key_to_index.keys

In [32]:
modelGoogle = gensim.models.Word2Vec(
                X_train["text"], 
                vector_size=300, 
                window=20,
                min_count=2, 
                sample=6e-5, 
                alpha=0.03, 
                min_alpha=0.0007, 
                negative=20,                                                    
                workers=7)
modelGoogle.build_vocab(X_train['text'])

In [33]:
training_examples_count = modelGoogle.corpus_count
training_examples_count

74960

In [35]:
modelGoogle.epochs

5

In [34]:
modelGoogle.wv.vectors_lockf = np.ones(len(modelGoogle.wv))

In [36]:
modelGoogle.build_vocab([google_wv.key_to_index], update=True)
modelGoogle.wv.intersect_word2vec_format("/content/ShardDrives/MyDrive/nlp/GoogleNews-vectors-negative300.bin.gz",binary=True, lockf=1.0)

In [37]:
modelGoogle.wv.most_similar(positive=["linux"])

[('osx', 0.7418608665466309),
 ('unix', 0.7179777026176453),
 ('filesystems', 0.6992602348327637),
 ('distro', 0.6973377466201782),
 ('slackware', 0.6968764662742615),
 ('distros', 0.6949489116668701),
 ('debian', 0.6920150518417358),
 ('mysql', 0.6869272589683533),
 ('kde', 0.6766390800476074),
 ('opensource', 0.6727070212364197)]

In [38]:
modelGoogle.train(X_train['text'],total_examples=training_examples_count, epochs=model.epochs)

(4613421, 11387685)

On constate que nos motes on des positionnement differents que notre propre modele

In [39]:
modelGoogle.wv.most_similar(positive=["linux"])

[('mint', 0.6779086589813232),
 ('arch', 0.6735519766807556),
 ('gnu', 0.6692314147949219),
 ('kernel', 0.6555151343345642),
 ('bsd', 0.644873321056366),
 ('unix', 0.6314297914505005),
 ('machine', 0.623516321182251),
 ('redhat', 0.6137810945510864),
 ('syscalls', 0.6130386590957642),
 ('distro', 0.6115866899490356)]

In [42]:
model.wv.most_similar(positive=["c#"])

[('vb', 0.8536680340766907),
 ('vbnet', 0.8349421620368958),
 ('msdn', 0.7928580641746521),
 ('net', 0.7539410591125488),
 ('ilspy', 0.7443284392356873),
 ('clr', 0.7141795754432678),
 ('il', 0.708362340927124),
 ('roslyn', 0.6944793462753296),
 ('skeet', 0.6742131114006042),
 ('c#net', 0.6717705130577087)]

In [44]:
model.wv.most_similar(positive=["database"])

[('db', 0.8428206443786621),
 ('oltp', 0.7693713307380676),
 ('rdbms', 0.7633110284805298),
 ('sql', 0.7512076497077942),
 ('nosql', 0.7313162684440613),
 ('dbms', 0.7286774516105652),
 ('record', 0.7282244563102722),
 ('phpmyadmin', 0.7086646556854248),
 ('mssql', 0.7073016166687012),
 ('mdb', 0.70614093542099)]

In [46]:
import numpy

def get_vect(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return numpy.zeros((model.vector_size,))

def sum_vectors(phrase, model):
    return sum(get_vect(w, model) for w in phrase)

def word2vec_features(X, model):
    feats = numpy.vstack([sum_vectors(p, model) for p in X])
    return feats

wv_train_feat2 = word2vec_features(X_train["text"], modelGoogle)
wv_train_feat2.shape

(74960, 300)

In [47]:
wv_test_feat2 = word2vec_features(X_test["text"], modelGoogle)
wv_test_feat2.shape

(18741, 300)

In [48]:

logisticWv_google =  Pipeline([
    #with word to vectorS
    ('std',StandardScaler()),
    ('test',OneVsRestClassifier(LogisticRegression(max_iter=1000))), 
])
logisticWv_google.fit(wv_train_feat2, y_train)
getMetrics(logisticWv_google,'logisticWv_google',wv_test_feat2, y_test)
# logisticWv.score(wv_test_feat3[0:100000], y_test[0:100000])

,estimator,accuracy,f1_macro,f1_micro,jaccard,recall,precision
0,logisticWv_google,0.31199,0.463,0.572,0.405,0.447,0.76


resultat moins performant 



<a name='wi'></a>

<div class="bg-primary text-lg-left text-white">
    <div >
        <h1 class="text-center" style="padding: 10px 0 10px 0;">
           intersection entre le vocab wikipedia externe et notre vocab
        </h1>
    </div>
</div>

In [ ]:
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors

In [49]:
wikipedia_wv = gensim.models.KeyedVectors.load_word2vec_format('/content/ShardDrives/MyDrive/nlp/glove-wiki-gigaword-300.gz')

In [ ]:
# google_wv.key_to_index.keys

In [50]:
modelWikipedia = gensim.models.Word2Vec(
                X_train["text"], 
                vector_size=300, 
                window=20,
                min_count=2, 
                sample=6e-5, 
                alpha=0.03, 
                min_alpha=0.0007, 
                negative=20,                                                    
                workers=7)
modelWikipedia.build_vocab(X_train['text'])

In [51]:
training_examples_count = modelWikipedia.corpus_count
training_examples_count

74960

In [52]:
modelWikipedia.epochs

5

In [53]:
modelWikipedia.wv.vectors_lockf = np.ones(len(modelWikipedia.wv))

In [ ]:
X_train.shape

(74960, 1)

In [54]:
modelWikipedia.build_vocab([wikipedia_wv.key_to_index], update=True)
modelWikipedia.wv.intersect_word2vec_format('/content/ShardDrives/MyDrive/nlp/glove-wiki-gigaword-300.gz', lockf=1.0)

In [55]:
modelWikipedia.predict_output_word('java')

[('client_secret', 3.520011e-05),
 ('editplus', 3.520011e-05),
 ('whatsnewkit', 3.520011e-05),
 ('portname', 3.520011e-05),
 ('operatorfunction', 3.520011e-05),
 ('masquerade', 3.520011e-05),
 ('alessandro', 3.520011e-05),
 ('scicomp', 3.520011e-05),
 ('npw', 3.520011e-05),
 ('gmdate', 3.520011e-05)]

In [56]:
modelWikipedia.wv.most_similar(positive=["linux"])

[('freebsd', 0.6575999855995178),
 ('unix', 0.654360830783844),
 ('os', 0.6241284608840942),
 ('ubuntu', 0.6010157465934753),
 ('solaris', 0.5962995886802673),
 ('kernel', 0.5935722589492798),
 ('desktop', 0.5804030895233154),
 ('debian', 0.5526514649391174),
 ('xp', 0.5510505437850952),
 ('server', 0.5466394424438477)]

In [57]:
modelWikipedia.train(X_train['text'],total_examples=training_examples_count, epochs=modelWikipedia.epochs)

(4613735, 11387685)

In [58]:
modelWikipedia.predict_output_word('java')

[('v', 0.99992275),
 ('hyper', 4.6564273e-05),
 ('express', 8.885369e-06),
 ('msbuild', 4.4354947e-06),
 ('resharper', 4.320347e-06),
 ('rc', 1.9827528e-06),
 ('omnisharp', 1.4687654e-06),
 ('tfs', 4.4820231e-07),
 ('hresult', 3.9514435e-07),
 ('intellisense', 3.8669197e-07)]

In [59]:
modelWikipedia.wv.most_similar(positive=["linux"])

[('gnu', 0.6421453952789307),
 ('arch', 0.6216177940368652),
 ('unix', 0.6140703558921814),
 ('kernel', 0.6005076169967651),
 ('machine', 0.558546245098114),
 ('window', 0.5448927283287048),
 ('fedora', 0.5448815226554871),
 ('mint', 0.5226677656173706),
 ('arm', 0.4994640350341797),
 ('syscalls', 0.4888329803943634)]

In [60]:
import numpy

def get_vect(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return numpy.zeros((model.vector_size,))

def sum_vectors(phrase, model):
    return sum(get_vect(w, model) for w in phrase)

def word2vec_features(X, model):
    feats = numpy.vstack([sum_vectors(p, model) for p in X])
    return feats

wv_train_feat3 = word2vec_features(X_train["text"], modelWikipedia)
wv_train_feat3.shape

(74960, 300)

In [61]:
wv_test_feat3 = word2vec_features(X_test["text"], modelWikipedia)
wv_test_feat3.shape

(18741, 300)

In [62]:
#training with vector size 50
logisticWvWiki =  Pipeline([
    #with word to vectorS
    ('std',StandardScaler()),
    ('test',OneVsRestClassifier(LogisticRegression(max_iter=1000))), 
])
logisticWvWiki.fit(wv_train_feat3, y_train)
getMetrics(logisticWvWiki,'logisticWvLoaded',wv_test_feat3, y_test)
# logisticWv.score(wv_test_feat3[0:100000], y_test[0:100000])

,estimator,accuracy,f1_macro,f1_micro,jaccard,recall,precision
0,logisticWvLoaded,0.33686,0.497,0.594,0.427,0.471,0.77


resultat presque equivalent pour la vocab wiki

nous allons le tester dans un gridSearch pour voir

<a name='co'></a>

<div class="bg-primary text-lg-left text-white">
    <div >
        <h1 class="text-center" style="padding: 10px 0 10px 0;">
           intersection entre le vocab conceptNet externe et notre vocab
        </h1>
    </div>
</div>

In [63]:
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors

In [64]:
conceptNet_wv = gensim.models.KeyedVectors.load_word2vec_format('/content/ShardDrives/MyDrive/nlp/conceptnet-numberbatch-17-06-300.gz')

In [65]:
# google_wv.key_to_index.keys

In [81]:
modelConceptNet = gensim.models.Word2Vec(
                X_train["text"], 
                vector_size=300, 
                window=20,
                min_count=2, 
                sample=6e-5, 
                alpha=0.03, 
                min_alpha=0.0007, 
                negative=20,                                                    
                workers=7)
modelConceptNet.build_vocab(X_train['text'])

In [82]:
training_examples_count = modelConceptNet.corpus_count
training_examples_count

74960

In [83]:
modelConceptNet.epochs

5

In [84]:
modelConceptNet.wv.vectors_lockf = np.ones(len(modelConceptNet.wv))

In [85]:
X_train.shape

(74960, 1)

In [86]:
modelConceptNet.build_vocab([conceptNet_wv.key_to_index], update=True)
modelConceptNet.wv.intersect_word2vec_format('/content/ShardDrives/MyDrive/nlp/conceptnet-numberbatch-17-06-300.gz', lockf=1.0)

In [87]:
modelConceptNet.predict_output_word('java')

[('client_secret', 3.520011e-05),
 ('editplus', 3.520011e-05),
 ('whatsnewkit', 3.520011e-05),
 ('portname', 3.520011e-05),
 ('operatorfunction', 3.520011e-05),
 ('masquerade', 3.520011e-05),
 ('alessandro', 3.520011e-05),
 ('scicomp', 3.520011e-05),
 ('npw', 3.520011e-05),
 ('gmdate', 3.520011e-05)]

In [88]:
modelConceptNet.wv.most_similar(positive=["linux"])

[('advantage', 0.6255552768707275),
 ('equinox', 0.6073939204216003),
 ('strip', 0.6062387228012085),
 ('assist', 0.589634120464325),
 ('parallax', 0.5736305117607117),
 ('webxml', 0.573511004447937),
 ('parsefacebookutils(pffacebooktokencachingstrategyo', 0.5663239359855652),
 ('banana', 0.5631721615791321),
 ('comappledt', 0.5575751662254333),
 ('preocess', 0.5457735061645508)]

In [89]:
modelConceptNet.train(X_train['text'],total_examples=training_examples_count, epochs=modelConceptNet.epochs)

(4612636, 11387685)

In [90]:
modelConceptNet.predict_output_word('java')

[('v', 0.9987191),
 ('resharper', 0.0007580151),
 ('rc', 7.1449394e-05),
 ('tfs', 5.1228828e-05),
 ('editor', 3.736896e-05),
 ('msbuild', 1.7247097e-05),
 ('ctp', 1.6567246e-05),
 ('xunit', 1.5118287e-05),
 ('winform', 1.0881047e-05),
 ('vscode', 9.390127e-06)]

In [91]:
modelConceptNet.wv.most_similar(positive=["linux"])

[('mint', 0.6334328055381775),
 ('gnu', 0.6171266436576843),
 ('syscalls', 0.6126101016998291),
 ('bsd', 0.6069719195365906),
 ('arch', 0.5932028293609619),
 ('kernel', 0.5917574763298035),
 ('distro', 0.5860258340835571),
 ('ld_library_path', 0.5574800372123718),
 ('fedora', 0.5474988222122192),
 ('sendmsg', 0.5462530851364136)]

In [92]:
import numpy

def get_vect(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return numpy.zeros((model.vector_size,))

def sum_vectors(phrase, model):
    return sum(get_vect(w, model) for w in phrase)

def word2vec_features(X, model):
    feats = numpy.vstack([sum_vectors(p, model) for p in X])
    return feats

wv_train_feat4 = word2vec_features(X_train["text"], modelConceptNet)
wv_train_feat4.shape

(74960, 300)

In [93]:
wv_test_feat4 = word2vec_features(X_test["text"], modelConceptNet)
wv_test_feat4.shape

(18741, 300)

In [94]:
#training with vector size 50
logisticWvWiki =  Pipeline([
    #with word to vectorS
    ('std',StandardScaler()),
    ('test',OneVsRestClassifier(LogisticRegression(max_iter=3000))), 
])
logisticWvWiki.fit(wv_train_feat4, y_train)
getMetrics(logisticWvWiki,'logisticWvLoaded',wv_test_feat4, y_test)
# logisticWv.score(wv_test_feat3[0:100000], y_test[0:100000])

,estimator,accuracy,f1_macro,f1_micro,jaccard,recall,precision
0,logisticWvLoaded,0.32426,0.476,0.581,0.413,0.457,0.765


resultat moins performant 



<a name='fa'></a>

<div class="bg-primary text-lg-left text-white">
    <div >
        <h1 class="text-center" style="padding: 10px 0 10px 0;">
          intersection entre le vocab fastText externe et notre vocab
        </h1>
    </div>
</div>


In [95]:
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors

In [102]:
fastText_wv = gensim.models.KeyedVectors.load_word2vec_format('/content/ShardDrives/MyDrive/nlp/fasttext-wiki-news-subwords-300.gz', encoding='utf-8')

In [ ]:
# google_wv.key_to_index.keys

In [103]:
modelFastTExt = gensim.models.Word2Vec(
                X_train["text"], 
                vector_size=300, 
                window=20,
                min_count=2, 
                sample=6e-5, 
                alpha=0.03, 
                min_alpha=0.0007, 
                negative=20,                                                    
                workers=7)
modelFastTExt.build_vocab(X_train['text'])

In [104]:
training_examples_count = modelFastTExt.corpus_count
training_examples_count

74960

In [105]:
modelConceptNet.epochs

5

In [106]:
modelFastTExt.wv.vectors_lockf = np.ones(len(modelFastTExt.wv))

In [107]:
X_train.shape

(74960, 1)

In [108]:
modelFastTExt.build_vocab([fastText_wv.key_to_index], update=True)
modelFastTExt.wv.intersect_word2vec_format('/content/ShardDrives/MyDrive/nlp/fasttext-wiki-news-subwords-300.gz', lockf=1.0)

In [109]:
modelFastTExt.predict_output_word('java')

[('client_secret', 3.520011e-05),
 ('editplus', 3.520011e-05),
 ('whatsnewkit', 3.520011e-05),
 ('portname', 3.520011e-05),
 ('operatorfunction', 3.520011e-05),
 ('masquerade', 3.520011e-05),
 ('alessandro', 3.520011e-05),
 ('scicomp', 3.520011e-05),
 ('npw', 3.520011e-05),
 ('gmdate', 3.520011e-05)]

In [110]:
modelFastTExt.wv.most_similar(positive=["linux"])

[('unix', 0.7261496186256409),
 ('distro', 0.7151504755020142),
 ('debian', 0.7010477185249329),
 ('cygwin', 0.6785581111907959),
 ('freebsd', 0.6748203635215759),
 ('redhat', 0.6732003688812256),
 ('distros', 0.655481219291687),
 ('vmware', 0.6455954909324646),
 ('userland', 0.6305261254310608),
 ('microsoft', 0.627495288848877)]

In [111]:
modelFastTExt.train(X_train['text'],total_examples=training_examples_count, epochs=modelFastTExt.epochs)

(4610149, 11387685)

In [112]:
modelFastTExt.predict_output_word('java')

[('v', 0.99856085),
 ('resharper', 0.0006579461),
 ('tfs', 9.2705974e-05),
 ('hyper', 7.158148e-05),
 ('csproj', 4.8017133e-05),
 ('f', 3.151934e-05),
 ('rc', 3.095811e-05),
 ('g', 2.6709506e-05),
 ('msbuild', 2.2571765e-05),
 ('express', 1.9417157e-05)]

In [113]:
modelFastTExt.wv.most_similar(positive=["linux"])

[('arch', 0.7498563528060913),
 ('gnu', 0.7277716994285583),
 ('unix', 0.7267122268676758),
 ('fedora', 0.6938449144363403),
 ('machine', 0.6929482817649841),
 ('kernel', 0.6762269735336304),
 ('arm', 0.6761000156402588),
 ('strace', 0.670507550239563),
 ('mint', 0.6571052074432373),
 ('filesystem', 0.6416885256767273)]

In [114]:
import numpy

def get_vect(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return numpy.zeros((model.vector_size,))

def sum_vectors(phrase, model):
    return sum(get_vect(w, model) for w in phrase)

def word2vec_features(X, model):
    feats = numpy.vstack([sum_vectors(p, model) for p in X])
    return feats

wv_train_feat5 = word2vec_features(X_train["text"], modelFastTExt)
wv_train_feat5.shape

(74960, 300)

In [115]:
wv_test_feat5 = word2vec_features(X_test["text"], modelFastTExt)
wv_test_feat5.shape

(18741, 300)

In [116]:
#training with vector size 50
logisticWvFastText =  Pipeline([
    #with word to vectorS
    ('std',StandardScaler()),
    ('test',OneVsRestClassifier(LogisticRegression(max_iter=3000))), 
])
logisticWvFastText.fit(wv_train_feat5, y_train)
getMetrics(logisticWvFastText,'logisticWvFastText',wv_test_feat5, y_test)
# logisticWv.score(wv_test_feat3[0:100000], y_test[0:100000])

,estimator,accuracy,f1_macro,f1_micro,jaccard,recall,precision
0,logisticWvFastText,0.28296,0.406,0.532,0.366,0.402,0.739


resultat pas performant du tout

A partir des resultats de notre benchmark , nous pouvons conclure que seul la vocab wikipedia est interessante a explorer, nous allons faire un gridseach dessus

<div class="bg-primary text-lg-left text-white">
    <div>
        <h1 class="text-center" style="padding: 10px 0 10px 0;">
          API disponlible
        </h1>
    </div>

</div>
    <div class="text-lg-left ">
       <a href="https://github.com/ismailazdad/stackoverflowTags" target="_blank">stackoverflow</a>
    </div>
     <div>       
       <a href="http://ismail2233.pythonanywhere.com/" target="_blank">online version</a>
    </div>

